# Custom NER based parser using Spacy3
1. Downloading appropriate the skeleton of base [config_file](https://spacy.io/usage/training) as per the system requirement.


2. Clone the source data [repo](https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git) to train the custom `ner` model.

    ```bash
    git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git
    ```

3. Generate the `config.cfg` file using the `base_config.cfg` i.e. used for coniguring the Spacy Model parameters.

    ```bash
    python -m spacy init fill-config base_config.cfg config.cfg
    ```



4. Parse and convert source training data into `*.spacy` format.


5. Train the blank `ner` based model on custom data using below command.

    ```bash
    python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id=0
    ```
    **Remark:** Here dev.spacy denotes the test data i.e. used for model evaluation


6. Evaluate model performance on Unseen data.

## 1. Installing dependencies & importing packages

In [1]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.1
    Uninstalling spacy-3.5.1:
      Successfully uninstalled spacy-3.5.1


In [2]:
!pip freeze > requirement.txt

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

After adding above 2 lines bash commands should work as expected

In [ ]:
# Checking GPU configuration
!nvidia-smi

Wed Apr 12 08:50:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    27W /  70W |    397MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Donloading and preparing the `base_config.cfg` file

In [ ]:
!python -m spacy init fill-config '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/data/base_config.cfg' '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/data/config.cfg'

2023-04-12 08:57:00.842471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume
Parser/data/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## 3. Loading the source data and preparing it for model training.

In [ ]:
cv_Data = json.load(open(r'/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/data/train_data.json','r'))

In [ ]:
len(cv_Data)

200

In [ ]:
cv_Data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
    
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        
        ents = []
        entity_indices = []
        
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity==True:
                continue

            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "    " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
            
    return db

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_Data, test_size=0.3)

In [ ]:
file = open(r"/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/error.txt", "w", encoding="utf-8")

db_train = get_spacy_doc(file, train)
db_train.to_disk(r'/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/train_data.spacy')

db_test = get_spacy_doc(file, test)
db_test.to_disk(r'/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 62.41it/s]


## 4. Model training

In [ ]:
!python -m spacy train '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/data/config.cfg' --output '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/output' --paths.train '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/train_data.spacy' --paths.dev '/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/test_data.spacy' --gpu-id=0

2023-04-12 09:12:45.287164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: /content/drive/MyDrive/Colab
Notebooks/Work/Gembo/NER based Resume Parser/output
ℹ Saving to output directory: /content/drive/MyDrive/Colab
Notebooks/Work/Gembo/NER based Resume Parser/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-12 09:12:56,039] [INFO] Set up nlp object from config
[2023-04-12 09:12:56,050] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-12 09:12:56,053] [INFO] Created vocabulary
[2023-04-12 09:12:56,054] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkp

## 5. Loading the base model and test the model on unseen data

In [ ]:
nlp = spacy.load(r'/content/drive/MyDrive/Colab Notebooks/Work/Gembo/NER based Resume Parser/output/model-best')

In [ ]:
doc = nlp('My name is Raj. I work as a ML Engineer at GEMBO. I have 2 years of experience')
for ent in doc.ents:
    print(ent.text, "------------>", ent.label_)

Raj. ------------> Name
ML Engineer ------------> Designation
2 years ------------> Years of Experience
